In [1]:
import pandas as pd
import spotipy
from sqlalchemy import create_engine
from secrets import spotify_secrets, localhost_db2, postgres_db
from spotipy.oauth2 import SpotifyOAuth
from queries import *

In [54]:
RANGES = {'short_term': 0, 'medium_term': 1, 'long_term': 2}
LIMIT = 50

def get_top_artists_df(sp):
    user_id = sp.me()['id']
    top_list = []
    for r in RANGES:
        top_artists = sp.current_user_top_artists(time_range=r, limit=LIMIT)
        for i, a in enumerate(top_artists['items']):
            this_top = {
                'user_id': user_id,
                'rank': i+1,
                'artist_id': a['id'],
                'timeframe': RANGES[r],
                'artist': a['name'],
                'genres': "; ".join(g for g in a['genres']),
                'artist_url': a['external_urls']['spotify'],
                'artist_image': a['images'][0]['url'],
                'popularity': a['popularity'],
            }
            top_list.append(this_top)
    return pd.DataFrame.from_dict(top_list)

def get_top_tracks_df(sp):
    user_id = sp.me()['id']
    top_list = []
    audio_features = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', \
                      'liveness', 'valence', 'tempo']
    for r in RANGES:
        top_tracks = sp.current_user_top_tracks(time_range=r, limit=LIMIT)
        for i, t in enumerate(top_tracks['items']):
            this_top = {
                'user_id': user_id,
                'rank': i+1,
                'track_id': t['id'],
                'timeframe': RANGES[r],
                'track_id': t['id'],
                'track': t['name'],
                'artists': "; ".join(a['name'] for a in t['artists']),
                'album': t['album']['name'],
                'album_image': t['album']['images'][0]['url'],
                'release_date': t['album']['release_date'],
                'track_url': t['external_urls']['spotify'],
                'timeframe': RANGES[r]
            }
            top_list.append(this_top)
    return pd.DataFrame.from_dict(top_list)

def top_to_dict(top_df, shuffle=False):
    top_dict = {}
    for i in range(3):
        this_top = top_df.loc[top_df['timeframe'] == i].to_dict('records')
        if shuffle:
            this_top = this_top
            random.shuffle(this_top)
        top_dict[i] = this_top
    return top_dict

In [34]:
DATABASE_URL = localhost_db2
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=spotify_secrets["Client Id"],
                                               client_secret=spotify_secrets["Client Secret"],
                                               redirect_uri="http://localhost:8892/callback",
                                               scope="user-top-read"))

In [ ]:
df_tt = get_top_tracks_df(sp)
df_tt.head()

In [ ]:
def get_music_features_df(sp, top_tracks):
    audio_features = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', \
                      'liveness', 'valence', 'tempo']
    df_music = pd.DataFrame()
    df_feature = pd.DataFrame(columns=['user_id'] + audio_features + ['timeframe'])
    user_id = top_tracks[2][0]['user_id']
    for timeframe in RANGES.values():
        this_feature = {'user_id': user_id, 'timeframe': timeframe}
        all_features = sp.audio_features([t['track_id'] for t in top_tracks[timeframe]])
        try:
            for f in audio_features:
                this_feature[f] = sum(a[f] for a in all_features) / len(all_features)
            df_feature = df_feature.append(this_feature, ignore_index=True)
            df_music = df_music.append(pd.DataFrame.from_dict(all_features))
        except:
            pass
    df_music = df_music[['id'] + audio_features + ['key', 'mode', 'duration_ms', 'time_signature']]
    df_music = df_music.rename(columns={'id': 'track_id'}).drop_duplicates()
    return df_feature, df_music

In [ ]:
top_tracks = top_to_dict(df_tt)
df_f, df_m = get_music_features_df(sp, top_tracks)
df_m.head()

In [ ]:
df_m.describe()

In [ ]:
engine = create_engine(DATABASE_URL)
df_m.to_sql('MusicFeatures', engine, index=False)

In [ ]:
df_t = pd.read_sql('select * from "Tracks" limit 50', engine)
df_t.head()

In [ ]:
audio_features = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', \
                      'liveness', 'valence', 'tempo']

all_features = sp.audio_features([t for t in df_t['track_id'].tolist()])
df_music = pd.DataFrame.from_dict(all_features)
df_music = df_music[['id'] + audio_features + ['key', 'mode', 'duration_ms', 'time_signature']]
df_music = df_music.rename(columns={'id': 'track_id'}).drop_duplicates()
df_music.head()

In [ ]:
df_music.to_sql('TempFeatures', engine, index=False, if_exists='replace')

In [ ]:
features_insert_query = """
INSERT INTO "MusicFeatures" (track_id, danceability, energy, loudness, speechiness, acousticness, instrumentalness,
    liveness, valence, tempo, key, mode, duration_ms, time_signature)
SELECT tf.track_id, tf.danceability, tf.energy, tf.loudness, tf.speechiness, tf.acousticness, tf.instrumentalness,
    tf.liveness, tf.valence, tf.tempo, tf.key, tf.mode, tf.duration_ms, tf.time_signature
FROM "TempFeatures" tf
ON CONFLICT (track_id) DO NOTHING
"""

In [ ]:
engine.execute(features_insert_query)

In [ ]:
df_users = pd.read_sql('select * from "Users"', engine)
df_users.head()

In [ ]:
audio_features = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', \
                  'liveness', 'valence', 'tempo']
df_music = pd.DataFrame()

for u in df_users['user_id']:
    df_t = pd.read_sql('select * from "TopTracks" where user_id = %(user_id)s', engine, params={'user_id': u})
    track_ids = df_t['track_id'].unique().tolist()
    for i in range(3):
        if i == 1 and len(track_ids) < 50:
            break
        elif i == 2 and len(track_ids) < 100:
            break
        t = track_ids[i*50:i*50+50]
        all_features = sp.audio_features(t)
        df_music = df_music.append(pd.DataFrame.from_dict(all_features))

df_music = df_music[['id'] + audio_features + ['key', 'mode', 'duration_ms', 'time_signature']]
df_music = df_music.rename(columns={'id': 'track_id'}).drop_duplicates()

In [ ]:
df_music

In [ ]:
df_music.to_sql('TempFeatures', engine, index=False, if_exists='replace')

In [ ]:
features_insert_query = """
INSERT INTO "MusicFeatures" (track_id, danceability, energy, loudness, speechiness, acousticness, instrumentalness,
    liveness, valence, tempo, key, mode, duration_ms, time_signature)
SELECT tf.track_id, tf.danceability, tf.energy, tf.loudness, tf.speechiness, tf.acousticness, tf.instrumentalness,
    tf.liveness, tf.valence, tf.tempo, tf.key, tf.mode, tf.duration_ms, tf.time_signature
FROM "TempFeatures" tf
ON CONFLICT (track_id) DO NOTHING
"""

In [ ]:
engine.execute(features_insert_query)

In [2]:
def get_user_profile(user_id):
    try:
        engine = create_engine(DATABASE_URL)
        df_user = pd.read_sql_query(user_query, engine, params={'user_id': user_id})
        user_profile = df_user.to_dict('records')[0]
        engine.dispose()
        return user_profile
    except:
        return None

def get_user_top(user_id):
    engine = create_engine(DATABASE_URL)
    df_a = get_top_artists(user_id, engine)
    df_t = get_top_tracks(user_id, engine)
    df_g = get_top_genres(user_id, engine)
    df_m = get_music_features(user_id, engine)
    engine.dispose()
    return df_a, df_t, df_g, df_m

def get_top_artists(user_id, engine):
    df = pd.read_sql_query(top_artists_query, engine, params={'user_id': user_id})
    return df

def get_top_tracks(user_id, engine):
    df = pd.read_sql_query(top_tracks_query, engine, params={'user_id': user_id})
    return df

def get_top_genres(user_id, engine):
    df = pd.read_sql_query(top_genres_query, engine, params={'user_id': user_id})
    return df

def get_music_features(user_id, engine):
    df = pd.read_sql_query(music_features_query, engine, params={'user_id': user_id})
    return df

In [3]:
DATABASE_URL = localhost_db2
df_a, df_t, df_g, df_m = get_user_top('12120382831')
df_m.head()

,user_id,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,timeframe
0,12120382831,0.55864,0.55286,-7.48940,0.081956,0.329089,0.078903,0.155022,0.361880,119.14190,0
1,12120382831,0.51484,0.64952,-6.51050,0.066170,0.234219,0.038204,0.157544,0.397502,125.81876,1
2,12120382831,0.65416,0.60960,-6.82908,0.143112,0.235778,0.034798,0.163644,0.425720,126.49528,2


In [4]:
top_dict = {}
is_percent = ['danceability', 'energy', 'acousticness', 'instrumentalness', 'liveness', 'valence']

for i in range(3):
    temp_dict = df_m.loc[df_m['timeframe'] == i].to_dict('records')[0]
    for p in is_percent:
        temp_dict[p] = round(temp_dict[p] *100)
    top_dict[i] = temp_dict

top_dict

{0: {'user_id': '12120382831',
  'danceability': 56,
  'energy': 55,
  'loudness': -7.489400000000001,
  'speechiness': 0.08195599999999999,
  'acousticness': 33,
  'instrumentalness': 8,
  'liveness': 16,
  'valence': 36,
  'tempo': 119.1419,
  'timeframe': 0},
 1: {'user_id': '12120382831',
  'danceability': 51,
  'energy': 65,
  'loudness': -6.5105,
  'speechiness': 0.06617,
  'acousticness': 23,
  'instrumentalness': 4,
  'liveness': 16,
  'valence': 40,
  'tempo': 125.81875999999998,
  'timeframe': 1},
 2: {'user_id': '12120382831',
  'danceability': 65,
  'energy': 61,
  'loudness': -6.82908,
  'speechiness': 0.14311199999999996,
  'acousticness': 24,
  'instrumentalness': 3,
  'liveness': 16,
  'valence': 43,
  'tempo': 126.49527999999998,
  'timeframe': 2}}

In [5]:
top_tracks_query = """
SELECT tt.rank, tt.track_id, tt.timeframe, t.track, t.artists, t.album, t.album_image, t.release_date, t.track_url,
       mf.danceability, mf.energy, mf.loudness, mf.acousticness, mf.instrumentalness, mf.liveness, mf.valence, mf.tempo
FROM "TopTracks" tt
JOIN "Tracks" t
ON tt.track_id = t.track_id
JOIN "MusicFeatures" mf
ON tt.track_id = mf.track_id
WHERE tt.user_id = %(user_id)s
ORDER BY tt.timeframe, tt.rank
"""

engine = create_engine(DATABASE_URL)
df_t = pd.read_sql_query(top_tracks_query, engine, params={'user_id': '12120382831'})
df_t.head()

,rank,track_id,timeframe,track,artists,album,album_image,release_date,track_url,danceability,energy,loudness,acousticness,instrumentalness,liveness,valence,tempo
0,1,2Kerz9H9IejzeIpjhDJoYG,0,Love,Lana Del Rey,Lust For Life,https://i.scdn.co/image/ab67616d0000b27395e2fd...,2017-07-21,https://open.spotify.com/track/2Kerz9H9IejzeIp...,0.527,0.366,-10.943,0.487000,0.002300,0.1100,0.234,98.994
1,2,73jVPicY2G9YHmzgjk69ae,0,Robbers,The 1975,The 1975,https://i.scdn.co/image/ab67616d0000b27304f21e...,2013-01-01,https://open.spotify.com/track/73jVPicY2G9YHmz...,0.621,0.692,-6.858,0.000363,0.000017,0.3350,0.381,99.806
2,3,2p8IUWQDrpjuFltbdgLOag,0,After Hours,The Weeknd,After Hours,https://i.scdn.co/image/ab67616d0000b2738863bc...,2020-03-20,https://open.spotify.com/track/2p8IUWQDrpjuFlt...,0.664,0.572,-6.099,0.081100,0.006040,0.1210,0.143,108.959
3,4,6ilc4vQcwMPlvAHFfsTGng,0,Sweet,Cigarettes After Sex,Cigarettes After Sex,https://i.scdn.co/image/ab67616d0000b27394d280...,2017-06-09,https://open.spotify.com/track/6ilc4vQcwMPlvAH...,0.450,0.511,-9.073,0.353000,0.766000,0.1390,0.115,96.563
4,5,6Vigp41BietH0WoFZ52JI5,0,All We Do,Oh Wonder,Oh Wonder,https://i.scdn.co/image/ab67616d0000b2737cc94e...,2015-09-04,https://open.spotify.com/track/6Vigp41BietH0Wo...,0.590,0.242,-11.724,0.978000,0.000124,0.0906,0.366,126.721


In [6]:
df_t.groupby('timeframe').mean()

,rank,danceability,energy,loudness,acousticness,instrumentalness,liveness,valence,tempo
timeframe,,,,,,,,,
0,25.5,0.55864,0.55286,-7.48940,0.329089,0.078903,0.155022,0.361880,119.14190
1,25.5,0.51484,0.64952,-6.51050,0.234219,0.038204,0.157544,0.397502,125.81876
2,25.5,0.65416,0.60960,-6.82908,0.235778,0.034798,0.163644,0.425720,126.49528


In [8]:
df_t.loc[df_t['timeframe'] == 0].mean().to_dict()

{'rank': 25.5,
 'timeframe': 0.0,
 'danceability': 0.5586400000000001,
 'energy': 0.5528599999999999,
 'loudness': -7.489400000000001,
 'acousticness': 0.32908893999999994,
 'instrumentalness': 0.07890282960000002,
 'liveness': 0.155022,
 'valence': 0.3618800000000001,
 'tempo': 119.1419}

In [13]:
top_tracks2_query = """
SELECT tt.user_id, tt.rank, tt.track_id, tt.timeframe, t.track, t.artists, t.album, t.album_image, t.release_date, t.track_url,
       mf.danceability, mf.energy, mf.loudness, mf.acousticness, mf.instrumentalness, mf.liveness, mf.valence, mf.tempo
FROM "TopTracks" tt
JOIN "Tracks" t
ON tt.track_id = t.track_id
JOIN "MusicFeatures" mf
ON tt.track_id = mf.track_id
WHERE tt.user_id in %(user_ids)s
ORDER BY tt.timeframe, tt.rank
"""

df_t = pd.read_sql_query(top_tracks2_query, engine, params={'user_ids': ('12120382831', 'chiazling')})
df_t.head()

,user_id,rank,track_id,timeframe,track,artists,album,album_image,release_date,track_url,danceability,energy,loudness,acousticness,instrumentalness,liveness,valence,tempo
0,12120382831,1,2Kerz9H9IejzeIpjhDJoYG,0,Love,Lana Del Rey,Lust For Life,https://i.scdn.co/image/ab67616d0000b27395e2fd...,2017-07-21,https://open.spotify.com/track/2Kerz9H9IejzeIp...,0.527,0.366,-10.943,0.487000,0.002300,0.110,0.234,98.994
1,chiazling,1,6Q7xEdFbvg3VJQ89SjIGpW,0,Running,Gen Neo,AD ASTRA,https://i.scdn.co/image/ab67616d0000b273695a5e...,2020-12-29,https://open.spotify.com/track/6Q7xEdFbvg3VJQ8...,0.588,0.326,-9.358,0.850000,0.000000,0.145,0.357,114.849
2,chiazling,2,1JsO8znEhoSGfOPwOiBJqF,0,Make Out to This,Jayvidd,Make Out to This,https://i.scdn.co/image/ab67616d0000b2733d3689...,2018-09-09,https://open.spotify.com/track/1JsO8znEhoSGfOP...,0.752,0.305,-8.960,0.913000,0.832000,0.282,0.574,151.911
3,12120382831,2,73jVPicY2G9YHmzgjk69ae,0,Robbers,The 1975,The 1975,https://i.scdn.co/image/ab67616d0000b27304f21e...,2013-01-01,https://open.spotify.com/track/73jVPicY2G9YHmz...,0.621,0.692,-6.858,0.000363,0.000017,0.335,0.381,99.806
4,12120382831,3,2p8IUWQDrpjuFltbdgLOag,0,After Hours,The Weeknd,After Hours,https://i.scdn.co/image/ab67616d0000b2738863bc...,2020-03-20,https://open.spotify.com/track/2p8IUWQDrpjuFlt...,0.664,0.572,-6.099,0.081100,0.006040,0.121,0.143,108.959


In [40]:
def get_music_features_df(sp, top_tracks):
    audio_features = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', \
                      'liveness', 'valence', 'tempo']
    dict_list = []
    track_ids = top_tracks['track_id'].unique().tolist()
    split = round(len(track_ids) / 3)
    for i in range(3):
        if i == 2:
            subset_ids = track_ids[i*split:]
        else:
            subset_ids = track_ids[i*split:i*split+split]
        this_features = sp.audio_features(subset_ids)
        dict_list.extend(this_features)
    df_feat = pd.DataFrame.from_dict(dict_list).rename(columns={'id': 'track_id'}).drop_duplicates()
    return df_feat

In [41]:
test = get_music_features_df(sp, df_t)
test

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,track_id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.527,0.366,10,-10.943,1,0.0277,0.487000,0.002300,0.1100,0.234,98.994,audio_features,2Kerz9H9IejzeIpjhDJoYG,spotify:track:2Kerz9H9IejzeIpjhDJoYG,https://api.spotify.com/v1/tracks/2Kerz9H9Iejz...,https://api.spotify.com/v1/audio-analysis/2Ker...,272743,4
1,0.588,0.326,11,-9.358,0,0.0366,0.850000,0.000000,0.1450,0.357,114.849,audio_features,6Q7xEdFbvg3VJQ89SjIGpW,spotify:track:6Q7xEdFbvg3VJQ89SjIGpW,https://api.spotify.com/v1/tracks/6Q7xEdFbvg3V...,https://api.spotify.com/v1/audio-analysis/6Q7x...,180149,3
2,0.752,0.305,1,-8.960,1,0.0589,0.913000,0.832000,0.2820,0.574,151.911,audio_features,1JsO8znEhoSGfOPwOiBJqF,spotify:track:1JsO8znEhoSGfOPwOiBJqF,https://api.spotify.com/v1/tracks/1JsO8znEhoSG...,https://api.spotify.com/v1/audio-analysis/1JsO...,160459,4
3,0.621,0.692,4,-6.858,1,0.0291,0.000363,0.000017,0.3350,0.381,99.806,audio_features,73jVPicY2G9YHmzgjk69ae,spotify:track:73jVPicY2G9YHmzgjk69ae,https://api.spotify.com/v1/tracks/73jVPicY2G9Y...,https://api.spotify.com/v1/audio-analysis/73jV...,254510,4
4,0.664,0.572,5,-6.099,0,0.0305,0.081100,0.006040,0.1210,0.143,108.959,audio_features,2p8IUWQDrpjuFltbdgLOag,spotify:track:2p8IUWQDrpjuFltbdgLOag,https://api.spotify.com/v1/tracks/2p8IUWQDrpju...,https://api.spotify.com/v1/audio-analysis/2p8I...,361027,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,0.569,0.507,0,-7.191,1,0.0239,0.128000,0.000000,0.0999,0.150,135.977,audio_features,4hYHaROtjB6YREh6AgW3fd,spotify:track:4hYHaROtjB6YREh6AgW3fd,https://api.spotify.com/v1/tracks/4hYHaROtjB6Y...,https://api.spotify.com/v1/audio-analysis/4hYH...,245200,4
240,0.449,0.513,6,-9.073,1,0.0280,0.344000,0.773000,0.1390,0.114,96.421,audio_features,2T6SuvI5GJXbJOCM1jweuW,spotify:track:2T6SuvI5GJXbJOCM1jweuW,https://api.spotify.com/v1/tracks/2T6SuvI5GJXb...,https://api.spotify.com/v1/audio-analysis/2T6S...,292110,4
241,0.724,0.647,1,-5.642,1,0.0658,0.280000,0.000000,0.1020,0.435,106.960,audio_features,2rPE9A1vEgShuZxxzR2tZH,spotify:track:2rPE9A1vEgShuZxxzR2tZH,https://api.spotify.com/v1/tracks/2rPE9A1vEgSh...,https://api.spotify.com/v1/audio-analysis/2rPE...,207333,4
242,0.469,0.780,10,-6.490,0,0.1220,0.224000,0.000000,0.1300,0.444,125.999,audio_features,1YmF9PvLhIISIANoMLIYGq,spotify:track:1YmF9PvLhIISIANoMLIYGq,https://api.spotify.com/v1/tracks/1YmF9PvLhIIS...,https://api.spotify.com/v1/audio-analysis/1YmF...,233061,4


In [15]:
df_t.groupby(['user_id', 'timeframe']).mean().reset_index()

,user_id,timeframe,rank,danceability,energy,loudness,acousticness,instrumentalness,liveness,valence,tempo
0,12120382831,0,25.5,0.55864,0.552860,-7.48940,0.329089,0.078903,0.155022,0.361880,119.14190
1,12120382831,1,25.5,0.51484,0.649520,-6.51050,0.234219,0.038204,0.157544,0.397502,125.81876
2,12120382831,2,25.5,0.65416,0.609600,-6.82908,0.235778,0.034798,0.163644,0.425720,126.49528
3,chiazling,0,25.5,0.67644,0.486438,-11.42330,0.483135,0.643930,0.149504,0.348762,123.32286
4,chiazling,1,25.5,0.62126,0.633800,-7.95950,0.282234,0.339657,0.159144,0.258318,125.85186
5,chiazling,2,25.5,0.58328,0.480140,-7.92996,0.416989,0.006333,0.154842,0.380780,117.91806


In [16]:
engine.dispose()

In [51]:
TF_WEIGHTS = {0: 3, 1: 2, 2: 1}

def get_most_features(df_t):
    to_keep = ['track_id', 'track', 'artists', 'album', 'album_image', 'track_url']
    overall_dict = {}
    for tf in TF_WEIGHTS.keys():
        this_dict = {'danceability': {}, 'energy': {}, 'loudness': {}, 'acousticness': {}, 'instrumentalness': {}, \
                     'liveness': {}, 'valence': {}, 'tempo': {}}
        df = df_t.loc[df_t['timeframe'] == tf]
        for feat in this_dict.keys():
            this_dict[feat]['min'] = df.loc[df[feat].idxmin()][to_keep].to_dict()
            this_dict[feat]['max'] = df.loc[df[feat].idxmax()][to_keep].to_dict()
        overall_dict[tf] = this_dict
    return overall_dict

In [45]:
this_dict = {'danceability': {}, 'energy': {}, 'loudness': {}, 'acousticness': {}, 'instrumentalness': {}, \
                     'liveness': {}, 'valence': {}, 'tempo': {}}
for feat in this_dict.keys():
        this_dict[feat]['min'] = 0
        this_dict[feat]['max'] = 1
this_dict

{'danceability': {'min': 0, 'max': 1},
 'energy': {'min': 0, 'max': 1},
 'loudness': {'min': 0, 'max': 1},
 'acousticness': {'min': 0, 'max': 1},
 'instrumentalness': {'min': 0, 'max': 1},
 'liveness': {'min': 0, 'max': 1},
 'valence': {'min': 0, 'max': 1},
 'tempo': {'min': 0, 'max': 1}}

In [52]:
get_most_features(df_t)

{0: {'danceability': {'min': {'track_id': '2rtGaCAeYtmcIvuZsvgTf6',
    'track': 'How to Disappear Completely',
    'artists': 'Radiohead',
    'album': 'Kid A',
    'album_image': 'https://i.scdn.co/image/ab67616d0000b273674c2b8b77e1e9259a2fcb87',
    'track_url': 'https://open.spotify.com/track/2rtGaCAeYtmcIvuZsvgTf6'},
   'max': {'track_id': '7m9OqQk4RVRkw9JJdeAw96',
    'track': 'Jocelyn Flores',
    'artists': 'XXXTENTACION',
    'album': '17',
    'album_image': 'https://i.scdn.co/image/ab67616d0000b273203c89bd4391468eea4cc3f5',
    'track_url': 'https://open.spotify.com/track/7m9OqQk4RVRkw9JJdeAw96'}},
  'energy': {'min': {'track_id': '5GUYJTQap5F3RDQiCOJhrS',
    'track': 'Self Control',
    'artists': 'Frank Ocean',
    'album': 'Blonde',
    'album_image': 'https://i.scdn.co/image/ab67616d0000b273c5649add07ed3720be9d5526',
    'track_url': 'https://open.spotify.com/track/5GUYJTQap5F3RDQiCOJhrS'},
   'max': {'track_id': '0w2kfnU1PFKxjmZFQ1J1X8',
    'track': 'Overthinking',
  

In [43]:
DATABASE_URL = localhost_db2

search_user_query = """
SELECT *
FROM "Users"
WHERE display_name ILIKE %(search)s
LIMIT 10
"""

search_artist_query = """
SELECT artist_id, artist, artist_url, artist_image
FROM "Artists"
WHERE artist ILIKE %(search)s
LIMIT 10
"""

search_track_query1 = """
SELECT track_id, track, artists, album, track_url, album_image
FROM "Tracks"
WHERE track ILIKE %(search)s
LIMIT 10
"""

search_track_query2 = """
SELECT track_id, track, artists, album, track_url, album_image
FROM "Tracks"
WHERE artists ILIKE %(search)s
LIMIT 10
"""

def search_database(query):
    search = '%' + query + '%'
    engine = create_engine(DATABASE_URL)
    df_u = pd.read_sql_query(search_user_query, engine, params={'search': search})
    df_a = pd.read_sql_query(search_artist_query, engine, params={'search': search})
    df_t = pd.read_sql_query(search_track_query1, engine, params={'search': search})
    if len(df_t) <= 10:
        df_t2 = pd.read_sql_query(search_track_query2, engine, params={'search': search})
        df_t = df_t.append(df_t2).iloc[:10]
    engine.dispose()
    return df_u, df_a, df_t

In [51]:
x, y, z = search_database('bin xuan')
x

,user_id,display_name,spotify_url,image_url,followers,date_created,last_updated,last_recommended
0,12120382831,Bin Xuan Kong,https://open.spotify.com/user/12120382831,https://scontent-hkt1-1.xx.fbcdn.net/v/t1.0-1/...,53,2021-01-05 16:01:39.106837,2021-01-21 20:44:59.364961,2021-01-21 20:45:04.267701


In [45]:
y

,artist_id,artist,artist_url,artist_image
0,00FQb4jTyendYWaN8pK0wa,Lana Del Rey,https://open.spotify.com/artist/00FQb4jTyendYW...,https://i.scdn.co/image/d79b3e6cc42de7f44c8f13...


In [46]:
z

,track_id,track,artists,album,track_url,album_image
0,5QqyRUZeBE04yJxsD1OC0I,Summer Bummer (feat. A$AP Rocky & Playboi Carti),Lana Del Rey; A$AP Rocky; Playboi Carti,Lust For Life,https://open.spotify.com/track/5QqyRUZeBE04yJx...,https://i.scdn.co/image/ab67616d0000b27395e2fd...
1,0fBSs3fRoh1yJcne77fdu9,Video Games - Remastered,Lana Del Rey,Born To Die - The Paradise Edition,https://open.spotify.com/track/0fBSs3fRoh1yJcn...,https://i.scdn.co/image/ab67616d0000b273ebc8cf...
2,5kMAugdkH2coRHRnW5FuDb,Born To Die,Lana Del Rey,Born To Die,https://open.spotify.com/track/5kMAugdkH2coRHR...,https://i.scdn.co/image/ab67616d0000b273aaa285...
3,1Ym6aMuT5bliaZMC67AmPp,Cherry,Lana Del Rey,Lust For Life,https://open.spotify.com/track/1Ym6aMuT5bliaZM...,https://i.scdn.co/image/ab67616d0000b27395e2fd...
4,6PnluwP0fjGnpIBsqTdUTq,Pretty When You Cry,Lana Del Rey,Ultraviolence (Deluxe),https://open.spotify.com/track/6PnluwP0fjGnpIB...,https://i.scdn.co/image/ab67616d0000b273162459...
5,5tyMJlMqaggzvuX7TtlrTe,Love song,Lana Del Rey,Norman Fucking Rockwell!,https://open.spotify.com/track/5tyMJlMqaggzvuX...,https://i.scdn.co/image/ab67616d0000b273db5832...
6,3RIgHHpnFKj5Rni1shokDj,Norman fucking Rockwell,Lana Del Rey,Norman Fucking Rockwell!,https://open.spotify.com/track/3RIgHHpnFKj5Rni...,https://i.scdn.co/image/ab67616d0000b273db5832...
7,2Kerz9H9IejzeIpjhDJoYG,Love,Lana Del Rey,Lust For Life,https://open.spotify.com/track/2Kerz9H9IejzeIp...,https://i.scdn.co/image/ab67616d0000b27395e2fd...
8,34rRFl0bz9PocxWuO2ca5J,Music To Watch Boys To,Lana Del Rey,Honeymoon,https://open.spotify.com/track/34rRFl0bz9PocxW...,https://i.scdn.co/image/ab67616d0000b273a3b3f4...
9,03hqMhmCZiNKMSPmVabPLP,Groupie Love (feat. A$AP Rocky),Lana Del Rey; A$AP Rocky,Lust For Life,https://open.spotify.com/track/03hqMhmCZiNKMSP...,https://i.scdn.co/image/ab67616d0000b27395e2fd...


In [47]:
x.to_dict('records')

[]

In [48]:
y.to_dict('records')

[{'artist_id': '00FQb4jTyendYWaN8pK0wa',
  'artist': 'Lana Del Rey',
  'artist_url': 'https://open.spotify.com/artist/00FQb4jTyendYWaN8pK0wa',
  'artist_image': 'https://i.scdn.co/image/d79b3e6cc42de7f44c8f131ed9b3ed594202e400'}]

In [49]:
z.to_dict('records')

[{'track_id': '5QqyRUZeBE04yJxsD1OC0I',
  'track': 'Summer Bummer (feat. A$AP Rocky & Playboi Carti)',
  'artists': 'Lana Del Rey; A$AP Rocky; Playboi Carti',
  'album': 'Lust For Life',
  'track_url': 'https://open.spotify.com/track/5QqyRUZeBE04yJxsD1OC0I',
  'album_image': 'https://i.scdn.co/image/ab67616d0000b27395e2fd1accb339fa14878190'},
 {'track_id': '0fBSs3fRoh1yJcne77fdu9',
  'track': 'Video Games - Remastered',
  'artists': 'Lana Del Rey',
  'album': 'Born To Die - The Paradise Edition',
  'track_url': 'https://open.spotify.com/track/0fBSs3fRoh1yJcne77fdu9',
  'album_image': 'https://i.scdn.co/image/ab67616d0000b273ebc8cfac8b586bc475b04918'},
 {'track_id': '5kMAugdkH2coRHRnW5FuDb',
  'track': 'Born To Die',
  'artists': 'Lana Del Rey',
  'album': 'Born To Die',
  'track_url': 'https://open.spotify.com/track/5kMAugdkH2coRHRnW5FuDb',
  'album_image': 'https://i.scdn.co/image/ab67616d0000b273aaa285930cd1623de3eb60c5'},
 {'track_id': '1Ym6aMuT5bliaZMC67AmPp',
  'track': 'Cherry',
